<a href="https://colab.research.google.com/github/MiniSoda17/comp3710-deep-learning/blob/main/COMP3710_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.3 DawnBench


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
# CIFAR-10 transforms
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])

# Datasets + Dataloaders
batch_size = 512
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
n_classes = len(classes)

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = None
        if stride != 1 or in_planes != planes:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, num_classes)
    def _make_layer(self, planes, blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_planes, planes, stride))
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_planes, planes))
        return nn.Sequential(*layers)
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

# def conv3x3(in_planes, out_planes, stride=1):
#     return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
#                      padding=1, bias=False)

# def conv1x1(in_planes, out_planes, stride=1):
#     return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

# class BasicBlock(nn.Module):
#     expansion = 1
#     def __init__(self, in_planes, planes, stride=1, dropout_rate=0.1):
#         super().__init__()
#         self.conv1 = conv3x3(in_planes, planes, stride)
#         self.bn1 = nn.BatchNorm2d(planes)
#         self.relu = nn.ReLU(inplace=True)
#         self.dropout1 = nn.Dropout2d(dropout_rate) if dropout_rate > 0 else nn.Identity()

#         self.conv2 = conv3x3(planes, planes)
#         self.bn2 = nn.BatchNorm2d(planes)
#         self.dropout2 = nn.Dropout2d(dropout_rate) if dropout_rate > 0 else nn.Identity()

#         self.downsample = None
#         if stride != 1 or in_planes != planes:
#             self.downsample = nn.Sequential(
#                 conv1x1(in_planes, planes, stride),
#                 nn.BatchNorm2d(planes)
#             )

#         # Improved weight initialization
#         self._init_weights()

#     def _init_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)

#     def forward(self, x):
#         identity = x

#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)
#         out = self.dropout1(out)

#         out = self.conv2(out)
#         out = self.bn2(out)
#         out = self.dropout2(out)

#         if self.downsample is not None:
#             identity = self.downsample(x)

#         out += identity
#         out = self.relu(out)
#         return out

# class ResNet18(nn.Module):
#     def __init__(self, num_classes=10, dropout_rate=0.1, use_mixup=False):
#         super().__init__()
#         self.in_planes = 64
#         self.use_mixup = use_mixup

#         # Enhanced initial conv with proper padding for CIFAR-10
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)

#         # ResNet layers with dropout
#         self.layer1 = self._make_layer(64, 2, stride=1, dropout_rate=dropout_rate)
#         self.layer2 = self._make_layer(128, 2, stride=2, dropout_rate=dropout_rate)
#         self.layer3 = self._make_layer(256, 2, stride=2, dropout_rate=dropout_rate)
#         self.layer4 = self._make_layer(512, 2, stride=2, dropout_rate=dropout_rate)

#         # Enhanced pooling and classification head
#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.dropout = nn.Dropout(0.5)  # Stronger dropout before final layer
#         self.fc = nn.Linear(512, num_classes)

#         # Initialize weights
#         self._init_weights()

#     def _init_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.Linear):
#                 nn.init.normal_(m.weight, 0, 0.01)
#                 nn.init.constant_(m.bias, 0)

#     def _make_layer(self, planes, blocks, stride, dropout_rate=0.1):
#         layers = []
#         layers.append(BasicBlock(self.in_planes, planes, stride, dropout_rate))
#         self.in_planes = planes
#         for _ in range(1, blocks):
#             layers.append(BasicBlock(self.in_planes, planes, dropout_rate=dropout_rate))
#         return nn.Sequential(*layers)

#     def forward(self, x):
#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)

#         out = self.layer1(out)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         out = self.layer4(out)

#         out = self.avgpool(out)
#         out = torch.flatten(out, 1)
#         out = self.dropout(out)
#         out = self.fc(out)
#         return out

# # Optional: Label Smoothing Loss for better generalization
# class LabelSmoothingCrossEntropy(nn.Module):
#     def __init__(self, smoothing=0.1):
#         super().__init__()
#         self.smoothing = smoothing

#     def forward(self, x, target):
#         confidence = 1. - self.smoothing
#         logprobs = F.log_softmax(x, dim=-1)
#         nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
#         nll_loss = nll_loss.squeeze(1)
#         smooth_loss = -logprobs.mean(dim=-1)
#         loss = confidence * nll_loss + self.smoothing * smooth_loss
#         return loss.mean()

# # Usage example:
# # model = ResNet18(num_classes=10, dropout_rate=0.1)
# # criterion = LabelSmoothingCrossEntropy(smoothing=0.1)  # Instead of nn.CrossEntropyLoss()


In [ ]:
model = ResNet18(num_classes=n_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.01, epochs=40, steps_per_epoch=len(trainloader)
)
scaler = GradScaler()

num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        running_loss += loss.item()
        progress_bar.set_postfix(loss=running_loss/len(trainloader))

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast():
                outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.2f}%")

torch.save(model.state_dict(), "/content/drive/MyDrive/resnet18_cifar10.pth")
print("Target reached. Model saved!")



/tmp/ipython-input-3969628217.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/40:   0%|          | 0/98 [00:00<?, ?it/s]/tmp/ipython-input-3969628217.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/40: 100%|██████████| 98/98 [00:05<00:00, 18.69it/s, loss=1.47]
/tmp/ipython-input-3969628217.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/40], Accuracy: 53.76%


Epoch 2/40: 100%|██████████| 98/98 [00:05<00:00, 18.08it/s, loss=1.04]


Epoch [2/40], Accuracy: 52.67%


Epoch 3/40: 100%|██████████| 98/98 [00:05<00:00, 18.60it/s, loss=0.832]


...

Epoch [38/40], Accuracy: 93.46%


Epoch 39/40: 100%|██████████| 98/98 [00:05<00:00, 18.83it/s, loss=0.00544]


Epoch [39/40], Accuracy: 93.48%


Epoch 40/40: 100%|██████████| 98/98 [00:05<00:00, 18.53it/s, loss=0.00477]


Epoch [40/40], Accuracy: 93.51%
Target reached. Model saved!


In [ ]:
model = ResNet18(num_classes=n_classes).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/resnet18_cifar10.pth", map_location=device))
print("Pretrained model loaded!")


Pretrained model loaded!


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scaler = GradScaler()

num_epochs = 1  # demo only
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(trainloader, desc=f"Demo Epoch {epoch+1}/{num_epochs}")
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item()
        progress_bar.set_postfix(loss=running_loss/len(trainloader))

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast():
                outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Demo Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.2f}%")


/tmp/ipython-input-3507590684.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Demo Epoch 1/1:   0%|          | 0/98 [00:00<?, ?it/s]/tmp/ipython-input-3507590684.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Demo Epoch 1/1: 100%|██████████| 98/98 [00:05<00:00, 18.48it/s, loss=0.0196]
/tmp/ipython-input-3507590684.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Demo Epoch [1/1], Accuracy: 92.99%


# 4.1 VAE


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import os
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import umap
from sklearn.preprocessing import StandardScaler

In [7]:
class CNNVAE(nn.Module):
    def __init__(self, latent_dim=32):
        super().__init__()
        self.latent_dim = latent_dim

        # Encoder
        self.encoder_conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=2, padding=1),  # 28 -> 14
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), # 14 -> 7
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1), # 7 -> 4
            nn.ReLU(),
            nn.Flatten(),
        )

        self.fc_mu = nn.Linear(128*4*4, latent_dim)
        self.fc_logvar = nn.Linear(128*4*4, latent_dim)

        # Decoder
        self.decoder_fc = nn.Linear(latent_dim, 128*4*4)

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1), # 4 -> 8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),  # 8 -> 16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 3, stride=2, padding=1, output_padding=1),   # 16 -> 32
            nn.Sigmoid(),
        )

    def encode(self, x):
        h = self.encoder_conv(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std if self.training else mu

    def decode(self, z):
        h = self.decoder_fc(z)
        h = h.view(-1, 128, 4, 4)
        return self.decoder_conv(h)[:, :, :28, :28]  # crop to 28×28

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def vae_loss_function(recon_x, x, mu, logvar, beta=1.0):
    # Reconstruction loss averaged per pixel
    BCE = nn.functional.binary_cross_entropy(
        recon_x, x, reduction="sum"
    ) / x.size(0)  # divide by batch size

    # KL divergence
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)

    return BCE + beta * KLD, BCE, KLD



In [8]:
# --- Data Loading ---
class OASISDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        # The glob.glob function finds all files matching a pattern
        # The '**' and 'recursive=True' find images in all subdirectories
        self.file_paths = glob.glob(os.path.join(root_dir, '**', '*.png'), recursive=True)
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert('L') # Convert to grayscale

        if self.transform:
            image = self.transform(image)

        return image

# Define transformations to resize and convert images to tensors
transformations = transforms.Compose([
    transforms.Resize((28, 28)), # Resize to 28x28
    transforms.ToTensor(),       # Convert to a PyTorch tensor (values between 0 and 1)
])

# Define paths on the Rangpur cluster
data_path = '/content/drive/My Drive/OASIS'
train_dir = os.path.join(data_path, 'keras_png_slices_train')
test_dir = os.path.join(data_path, 'keras_png_slices_test')

# Create a dataset and a data loader for the training data
train_dataset = OASISDataset(root_dir=train_dir, transform=transformations)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [ ]:
# --- Training Loop ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = CNNVAE(latent_dim=32).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 50 # Increase this for better results

print("Starting VAE training...")

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()

        recon_batch, mu, logvar = model(data)
        loss, _, _ = vae_loss_function(recon_batch, data, mu, logvar)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}/{len(train_loader)}, Loss: {loss.item() / len(data):.4f}')

    avg_loss = train_loss / len(train_loader.dataset)
    print(f'Epoch: {epoch+1}, Average Loss: {avg_loss:.4f}')

print("Training complete.")
torch.save(model.state_dict(), 'vae_oasis.pth')

In [ ]:
# --- Latent Manifold Visualization (Sampling) ---
model.eval()
print("Generating latent space visualization by sampling...")

# Create a 2D sampling grid from a 2D slice of the 32D latent space
n = 10  # Number of images per row/column
grid_x = np.linspace(-3, 3, n)
grid_y = np.linspace(-3, 3, n)

figure = np.zeros((28 * n, 28 * n))
latent_dim = 32

with torch.no_grad():
    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z = torch.zeros(1, latent_dim).to(device)
            z[0, 0] = xi  # Sample along the first latent dimension
            z[0, 1] = yi  # Sample along the second latent dimension

            reconstruction = model.decode(z).cpu()
            img = reconstruction.view(28, 28).numpy()
            figure[i * 28: (i + 1) * 28, j * 28: (j + 1) * 28] = img

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='gray')
plt.title('VAE Latent Manifold Visualization (Sampling from 2D Slice)')
plt.axis('off')
plt.savefig('vae_manifold_sampling.png')
plt.show()

# --- Latent Manifold Visualization (UMAP) ---
print("Generating latent space visualization using UMAP...")

# Collect latent vectors from the test set
test_dataset = OASISDataset(root_dir=test_dir, transform=transformations)
test_loader = DataLoader(test_dataset, batch_size=128)

latents = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        mu, _ = model.encode(data)
        latents.append(mu.cpu().numpy())

latents = np.concatenate(latents, axis=0)
latents = StandardScaler().fit_transform(latents)

# Reduce dimensionality to 2 using UMAP
reducer = umap.UMAP(n_components=2, random_state=42)
embedding = reducer.fit_transform(latents)

plt.figure(figsize=(10, 8))
plt.scatter(embedding[:, 0], embedding[:, 1], s=1, alpha=0.5)
plt.title('UMAP Projection of VAE Latent Space')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.savefig('vae_manifold_umap.png')
plt.show()

# 4.2 UNET

Using device: cuda
Train data path: /content/drive/OASIS/keras_png_slices_train
Validation data path: /content/drive/OASIS/keras_png_slices_validate
No PNG images found in /content/drive/OASIS/keras_png_slices_train. Please check the path.


NameError: name 'train_loader' is not defined

# 4.3 GAN